# Extraction Notebook

This notebook steps will help to access the gz raw datasets from IMDB, store them in the 'extracted' folder, filter 
them and store them in the folder "clean" in order to access them later.

## Extraction Steps

In [54]:
import os
import gzip
import pandas as pd
import numpy as np

In [55]:
# Specify directories
base_folder = os.getcwd()[:-10]
raw_dir = os.path.join(base_folder, 'raw')
extracted_dir = os.path.join(base_folder, 'extracted')
clean_dir = os.path.join(base_folder, 'clean')

In [56]:
files_zipped =['title.ratings.tsv.gz', 'title.basics.tsv.gz', 'title.crew.tsv.gz', 'name.basics.tsv.gz']

In [57]:
# First check if there is already data downloaded
files_in_folder = os.listdir(extracted_dir)

if 'title.ratings.tsv' in files_in_folder and 'title.basics.tsv' in files_in_folder \
    and 'title.crew.tsv' in files_in_folder and 'name.basics.tsv' in files_in_folder:
    df_ratings = pd.read_csv(os.path.join(extracted_dir, 'title.ratings.tsv'), sep= '\t')
    df_basics = pd.read_csv(os.path.join(extracted_dir, 'title.basics.tsv'), sep= '\t')
    df_crew = pd.read_csv(os.path.join(extracted_dir, 'title.crew.tsv'), sep= '\t')
    df_names = pd.read_csv(os.path.join(extracted_dir, 'name.basics.tsv'), sep= '\t')

 # If the data has not been downloaded, perform extraction
else:
    for file in files_zipped:
        # Extract zipped content
        gzipped_file_path = os.path.join(base_folder, raw_dir, file)

        with gzip.open(gzipped_file_path, 'rt') as gzipped_file:
        # Specify the path to the extracted TSV file (remove '.gz' extension)
            extracted_file_path = os.path.join(extracted_dir, file[:-3])

        # Write the extracted content to the TSV file
            with open(extracted_file_path, 'w') as extracted_file:
                extracted_file.write(gzipped_file.read())

            if file == 'title.ratings.tsv.gz':
                df_ratings = pd.read_csv(extracted_file_path, sep='\t')
            elif file == 'title.crew.tsv.gz':
                df_crew = pd.read_csv(extracted_file_path, sep='\t')
            elif file == 'name.basics.tsv.gz':
                df_names = pd.read_csv(extracted_file_path, sep='\t')
            else:
                df_basics = pd.read_csv(extracted_file_path, sep='\t')


/var/folders/7t/j9_7cdrd5594tqknpkvcx2gr0000gn/T/ipykernel_21335/2409993370.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_basics = pd.read_csv(os.path.join(extracted_dir, 'title.basics.tsv'), sep= '\t')


## Merging Steps

### Basics File

In [58]:
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [59]:
# Consider just movies
df_basics = df_basics[df_basics['titleType']=='movie']

In [60]:
# Drop Original Title and end year
df_basics.drop(columns=['originalTitle', 'endYear'], inplace= True)

In [61]:
# Drop rows with Start year, run time minutes and genres null
df_basics = df_basics[df_basics['startYear']!="\\N"]
df_basics = df_basics[df_basics['runtimeMinutes']!="\\N"]
df_basics = df_basics[df_basics['genres']!="\\N"]

In [62]:
# Convert startYear, runtimeMinutes  columns to integer
df_basics['startYear']=df_basics['startYear'].astype('int32')
df_basics['runtimeMinutes']=df_basics['runtimeMinutes'].astype('int32')

In [63]:
# Consider just movies filmed between 1970 and 2022
df_basics= df_basics[(df_basics['startYear']>1970) & (df_basics['startYear']<2023)]

In [64]:
# For cases with more than one genre, keep the first
df_basics['genres'] = df_basics['genres'].apply(lambda x: x.split(',')[0] if ',' in str(x) else x)

### Crew File

In [65]:
df_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [66]:
# Drop Writers
df_crew.drop(columns=['writers'], inplace= True)

### Names File

In [67]:
df_names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0027125,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0075213,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0056404,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0077975,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0083922,tt0050976"


In [68]:
# Rename column of directors for later merge
df_names = df_names.rename(columns={'nconst':'directors'})

In [69]:
df_names.drop(columns=['deathYear', 'primaryProfession', 'knownForTitles'], inplace= True)

In [70]:
df_names.head()

,directors,primaryName,birthYear
0,nm0000001,Fred Astaire,1899
1,nm0000002,Lauren Bacall,1924
2,nm0000003,Brigitte Bardot,1934
3,nm0000004,John Belushi,1949
4,nm0000005,Ingmar Bergman,1918


### Merging crew and names file

In [71]:
df_crew= df_crew.merge(df_names, how='left', on='directors')

In [72]:
df_crew.head()

,tconst,directors,primaryName,birthYear
0,tt0000001,nm0005690,William K.L. Dickson,1860
1,tt0000002,nm0721526,Émile Reynaud,1844
2,tt0000003,nm0721526,Émile Reynaud,1844
3,tt0000004,nm0721526,Émile Reynaud,1844
4,tt0000005,nm0005690,William K.L. Dickson,1860


In [73]:
df_crew.iloc[-1]

tconst                   tt9916880
directors      nm0584014,nm0996406
primaryName                    NaN
birthYear                      NaN
Name: 10547109, dtype: object

In [74]:
# For cases with more than one director, keep the first
df_crew['directors'] = df_crew['directors'].apply(lambda x: x.split(',')[0] if ',' in str(x) else x)

### Merging Basics, Ratings and Crew Files

In [75]:
len(df_basics)

306532

In [76]:
df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2014
1,tt0000002,5.7,270
2,tt0000003,6.5,1937
3,tt0000004,5.5,178
4,tt0000005,6.2,2712


In [77]:
df_ratings.dtypes

tconst            object
averageRating    float64
numVotes           int64
dtype: object

In [78]:
df_merge = df_basics.merge(df_ratings, how='left', on='tconst')

In [79]:
len(df_merge)

306532

In [80]:
df_merge = df_merge.merge(df_crew, how='left', on='tconst')

In [81]:
df_merge.head()

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,primaryName,birthYear
0,tt0013274,movie,Istoriya grazhdanskoy voyny,0,2021,94,Documentary,6.8,63.0,nm0412842,NaN,NaN
1,tt0015724,movie,Dama de noche,0,1993,102,Drama,6.1,28.0,nm0529960,Eva López Sánchez,1954
2,tt0028248,movie,Shipmates o' Mine,0,2022,87,Musical,4.2,42.0,nm0593632,Oswald Mitchell,1897
3,tt0035423,movie,Kate & Leopold,0,2001,118,Comedy,6.4,88385.0,nm0003506,James Mangold,1963
4,tt0036606,movie,"Another Time, Another Place",0,1983,118,Drama,6.4,350.0,nm0705535,Michael Radford,1946


## Data Cleaning

### Duplicates

In [82]:
df_merge.duplicated().sum()

0

### Missing Values

In [83]:
# Converting "\\N" values in Null values so that we can rack them together
df_merge.replace("\\N", np.nan, inplace=True)

In [84]:
df_merge.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
isAdult                0
startYear              0
runtimeMinutes         0
genres                 0
averageRating      96082
numVotes           96082
directors           6309
primaryName        37971
birthYear         195322
dtype: int64

In [85]:
df_merge.isnull().sum().sort_values(ascending=False)/len(df_merge)

birthYear         0.637199
averageRating     0.313449
numVotes          0.313449
primaryName       0.123873
directors         0.020582
tconst            0.000000
titleType         0.000000
primaryTitle      0.000000
isAdult           0.000000
startYear         0.000000
runtimeMinutes    0.000000
genres            0.000000
dtype: float64

**Handling birth year missing entries**  

63.7% of missing data -> drop the feature

In [86]:
df_merge.drop(columns=['birthYear'], inplace=True)

**Rating and votes**

In [87]:
df_merge[df_merge['averageRating'].isnull()]

,tconst,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,primaryName
25,tt0062896,movie,Divorcio a la andaluza,0,1975,79,Comedy,NaN,NaN,nm0951373,José María Zabalza
35,tt0063987,movie,Living on Credit,0,1972,87,Documentary,NaN,NaN,nm0016105,Gustavo Alatriste
40,tt0064430,movie,Hildur and the Magician,0,1971,95,Fantasy,NaN,NaN,nm0430071,Larry Jordan
133,tt0065829,movie,Hembra,0,1971,117,Crime,NaN,NaN,nm0273202,César Fernández Ardavín
145,tt0065939,movie,Khozyain,0,1971,99,Drama,NaN,NaN,nm0947671,Mikhail Yershov
...,...,...,...,...,...,...,...,...,...,...,...
306521,tt9916162,movie,Making of 'La Virgen de los sicarios',0,1999,45,Documentary,NaN,NaN,nm0652213,Luis Ospina
306523,tt9916186,movie,Illenau - die Geschichte einer ehemaligen Heil...,0,2017,84,Documentary,NaN,NaN,nm10538274,Frank König
306528,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,Documentary,NaN,NaN,nm9272490,NaN
306529,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary,NaN,NaN,nm0652213,Luis Ospina


In [88]:
df_merge[df_merge['averageRating'].isnull()]['startYear'].value_counts()

2016    5079
2017    5012
2015    4970
2022    4937
2018    4931
2014    4838
2021    4765
2019    4580
2013    4560
2012    4282
2020    4079
2011    3822
2010    3719
2009    3183
2008    2662
2007    2148
2006    1929
2005    1753
2004    1477
2003    1298
2002    1054
2001     974
2000     912
1990     857
1997     787
1998     780
1989     780
1988     768
1999     755
1987     733
1992     730
1995     712
1991     711
1986     696
1996     690
1985     685
1984     680
1983     663
1994     662
1980     644
1981     640
1982     615
1976     596
1993     585
1975     580
1978     571
1971     557
1979     544
1974     540
1972     527
1977     519
1973     511
Name: startYear, dtype: int64

In [89]:
avg_ratings = df_merge.groupby('startYear')[['averageRating']].mean()
avg_ratings.tail(10)

,averageRating
startYear,
2013,6.241269
2014,6.259270
2015,6.229540
2016,6.236882
2017,6.206300
2018,6.141048
2019,6.155435
2020,6.123368
2021,6.197894


In [90]:
# Custom function to replace null values (in ratings) with rating average of the specific year
def custom_imputer(row):
    year = row['startYear']
    if pd.isnull(row['averageRating']):
        return float(avg_ratings[avg_ratings.index== year]['averageRating'])

    else:
        return row['averageRating']

In [91]:
df_merge['averageRating']=df_merge.apply(custom_imputer, axis=1)
df_merge.isnull().sum()

tconst                0
titleType             0
primaryTitle          0
isAdult               0
startYear             0
runtimeMinutes        0
genres                0
averageRating         0
numVotes          96082
directors          6309
primaryName       37971
dtype: int64

In [92]:
avg_votes = df_merge.groupby('startYear')[['numVotes']].mean()
avg_votes.tail(10)

,numVotes
startYear,
2013,5679.094213
2014,5378.605819
2015,4123.209011
2016,4262.084187
2017,3728.366967
2018,3403.999581
2019,3529.690639
2020,2231.438571
2021,3106.371305


In [93]:
# Custom function to replace null values (in votes) with votes average of the specific year
def custom_imputer_2(row):
    year = row['startYear']
    if pd.isnull(row['numVotes']):
        return float(avg_votes[avg_votes.index== year]['numVotes'])

    else:
        return row['numVotes']

In [94]:
df_merge['numVotes']=df_merge.apply(custom_imputer_2, axis=1)
df_merge.isnull().sum()

tconst                0
titleType             0
primaryTitle          0
isAdult               0
startYear             0
runtimeMinutes        0
genres                0
averageRating         0
numVotes              0
directors          6309
primaryName       37971
dtype: int64

**Directos and Primary Name**  

Directors column can be dropped since its just a key we used to merge the information.  
Primary Names can be left with null values, it will just be a helper to search movies.

In [95]:
df_merge.drop(columns=['directors'], inplace=True)
df_merge.rename(columns={'primaryName': 'Director'}, inplace=True)

In [96]:
# Save to csv
df_merge.to_csv(path_or_buf=os.path.join(clean_dir,'clean_df.csv'), index=False)